In [ ]:
import SimpleITK as sitk
import scipy.ndimage
import numpy as np
import os
import shutil

In [ ]:
def resample_spacing(image, spacing, new_spacing):
    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor,order=0)

    return image

In [ ]:
out_path = '../output'
path = '../segdl'
for folder in os.listdir(path):
    for file in os.listdir(os.path.join(path,folder)):
        img_path = os.path.join(path,folder,file,'CT_low.nii.gz')
        mask_path = os.path.join(out_path,file,'PT_FDG_Tumor_PT_0000.nii.gz')
        # print(img_path,mask_path)
        img_itk = sitk.ReadImage(img_path)
        img = sitk.GetArrayFromImage(img_itk)
        mask_itk = sitk.ReadImage(mask_path)
        mask = sitk.GetArrayFromImage(mask_itk)
        spacing_pet = np.array(img_itk.GetSpacing())
        spacing_mask = np.array(mask_itk.GetSpacing())
        new_spacing = np.array([spacing_pet[2],spacing_pet[1],spacing_pet[0]], dtype=np.float32)
        spacing = np.array([spacing_mask[2],spacing_mask[1],spacing_mask[0]], dtype=np.float32)
        resampled_mask= resample_spacing(mask, spacing, new_spacing)
        c1 = int(abs((resampled_mask.shape[0]-img.shape[0])/2))
        c2 = int(abs((resampled_mask.shape[0]+img.shape[0])/2))
        h1 = abs((resampled_mask.shape[1]-img.shape[1])/2)
        h2 = abs((resampled_mask.shape[1]+img.shape[1])/2)
        w1 = abs((resampled_mask.shape[2]-img.shape[2])/2)
        w2 = abs((resampled_mask.shape[2]+img.shape[2])/2)
        if mask.shape[1] == 144:
            if h1-int(h1) == 0 and c1 == 0:
                mask_re = np.pad(resampled_mask,((c1,c1),(int(h1),int(h1)),(int(w1),int(w1))),"constant")
            elif h1-int(h1) != 0 and c1 == 0:
                mask_re = np.pad(resampled_mask,((c1,c1),(int(h1)+1,int(h1)),(int(w1)+1,int(w1))),"constant")
            else:
                print("c1!=0",file)
        if mask.shape[1] ==200:
            if c1 == 0:
                mask_re = resampled_mask[c1:c2,int(h1):int(h2),int(w1):int(w2)]
            else:
                print("c1!=0",file)
        if mask_re.shape != img.shape:
            print("!!!",file)
        output_path = os.path.join(out_path,file,'CT_M.nii.gz')
        mask_img = sitk.GetImageFromArray((mask_re))
        sitk.WriteImage(mask_img, output_path)